# Module 2.4: Volume and Volatility Indicators

**Topics:** Volume Profile, Bollinger Bands, ATR, VIX analysis
**Key concepts:** Volume analysis, volatility measurement, multi-timeframe analysis

This notebook explores volume and volatility indicators that provide crucial insights into market dynamics beyond just price movement. Volume confirms price action, while volatility indicators help assess market risk and potential breakouts.

## 1. Introduction to Volume and Volatility Analysis

Volume and volatility are two fundamental dimensions of market analysis:

**Volume Analysis:**
- Confirms the strength of price movements
- Identifies accumulation and distribution phases
- Reveals institutional activity

**Volatility Analysis:**
- Measures price uncertainty and risk
- Identifies periods of expansion and contraction
- Helps with position sizing and risk management

Together, these indicators provide a more complete picture of market dynamics than price alone.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import requests
import os
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

## 2. Data Preparation

Let's fetch data for multiple timeframes to demonstrate multi-timeframe analysis.

In [2]:
# Setup proxy and fetch data
symbol = 'AAPL'
start_date = '2024-01-01'
end_date = datetime.now().strftime("%Y-%m-%d")

# Proxy configuration
session = requests.Session()
proxy_host = "localhost"
proxy_port = 10809
proxy_url = f"http://{proxy_host}:{proxy_port}"
session.proxies.update({
    "http": proxy_url,
    "https": proxy_url,
})
os.environ['HTTP_PROXY'] = proxy_url
os.environ['HTTPS_PROXY'] = proxy_url

ticker = yf.Ticker(symbol)
ticker.session = session

# Download daily data
data = ticker.history(start=start_date, end=end_date, interval='1d')
data = data.round(2)

print(f"Downloaded {len(data)} days of data for {symbol}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print("\nData columns:", data.columns.tolist())
data.head()

Downloaded 385 days of data for AAPL
Date range: 2024-01-02 to 2025-07-16

Data columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-02 00:00:00-05:00,185.79,187.07,182.55,184.29,82488700,0.0,0.0
2024-01-03 00:00:00-05:00,182.88,184.53,182.10,182.91,58414500,0.0,0.0
2024-01-04 00:00:00-05:00,180.83,181.76,179.57,180.59,71983600,0.0,0.0
2024-01-05 00:00:00-05:00,180.67,181.43,178.86,179.86,62303300,0.0,0.0
2024-01-08 00:00:00-05:00,180.77,184.25,180.18,184.21,59144500,0.0,0.0


## 3. Volume Analysis

### 3.1 Volume Moving Averages

Volume moving averages help identify periods of unusual trading activity.

In [3]:
# Calculate volume indicators
def calculate_volume_indicators(data, short_window=10, long_window=50):
    """
    Calculate various volume-based indicators
    """
    # Volume moving averages
    data['Volume_SMA_10'] = data['Volume'].rolling(window=short_window).mean()
    data['Volume_SMA_50'] = data['Volume'].rolling(window=long_window).mean()
    
    # Volume ratio (current volume vs average)
    data['Volume_Ratio'] = data['Volume'] / data['Volume_SMA_50']
    
    # On-Balance Volume (OBV)
    data['Price_Change'] = data['Close'].diff()
    data['OBV'] = (np.sign(data['Price_Change']) * data['Volume']).fillna(0).cumsum()
    
    # Volume Weighted Average Price (VWAP)
    data['VWAP'] = (data['Volume'] * (data['High'] + data['Low'] + data['Close']) / 3).cumsum() / data['Volume'].cumsum()
    
    return data

data = calculate_volume_indicators(data)
print("Volume indicators calculated successfully!")

Volume indicators calculated successfully!


### 3.2 Volume Profile Analysis

Volume profile shows the volume traded at each price level, revealing key support and resistance zones.

In [4]:
def calculate_volume_profile(data, bins=50):
    """
    Calculate volume profile - volume distribution across price levels
    """
    # Get price range
    price_min = data['Low'].min()
    price_max = data['High'].max()
    
    # Create price bins
    price_bins = np.linspace(price_min, price_max, bins)
    
    # Calculate volume for each price level
    volume_profile = []
    
    for i in range(len(price_bins) - 1):
        # Check which candles fall within this price range
        mask = (data['Low'] <= price_bins[i+1]) & (data['High'] >= price_bins[i])
        volume_in_range = data.loc[mask, 'Volume'].sum()
        
        volume_profile.append({
            'price_level': (price_bins[i] + price_bins[i+1]) / 2,
            'volume': volume_in_range
        })
    
    volume_profile_df = pd.DataFrame(volume_profile)
    
    # Find Point of Control (POC) - price level with highest volume
    poc_idx = volume_profile_df['volume'].idxmax()
    poc_price = volume_profile_df.loc[poc_idx, 'price_level']
    
    # Find Value Area (70% of volume)
    total_volume = volume_profile_df['volume'].sum()
    target_volume = total_volume * 0.7
    
    # Sort by volume and find 70% range
    sorted_profile = volume_profile_df.sort_values('volume', ascending=False)
    cumulative_volume = 0
    value_area_prices = []
    
    for _, row in sorted_profile.iterrows():
        cumulative_volume += row['volume']
        value_area_prices.append(row['price_level'])
        if cumulative_volume >= target_volume:
            break
    
    value_area_high = max(value_area_prices)
    value_area_low = min(value_area_prices)
    
    return volume_profile_df, poc_price, value_area_high, value_area_low

# Calculate volume profile for recent data (last 3 months)
recent_data = data.tail(90)  # Last 90 days
volume_profile, poc_price, va_high, va_low = calculate_volume_profile(recent_data)

print(f"Point of Control (POC): ${poc_price:.2f}")
print(f"Value Area High: ${va_high:.2f}")
print(f"Value Area Low: ${va_low:.2f}")

Point of Control (POC): $200.61
Value Area High: $213.84
Value Area Low: $185.90


## 4. Bollinger Bands

Bollinger Bands consist of a middle line (moving average) and two outer bands that are standard deviations away from the middle line. They help identify overbought/oversold conditions and potential breakouts.

### Formula
Bollinger Bands are calculated as follows:
$$
\text{Middle Band} = \text{SMA}(\text{Close}, 20)
$$
$$
\text{Upper Band} = \text{Middle Band} + (2 \times \sigma)
$$
$$
\text{Lower Band} = \text{Middle Band} - (2 \times \sigma)
$$
Where $\sigma$ is the standard deviation of the closing price over the same period.

In [5]:
def calculate_bollinger_bands(data, window=20, num_std=2):
    """
    Calculate Bollinger Bands
    """
    # Middle band (SMA)
    data['BB_Middle'] = data['Close'].rolling(window=window).mean()
    
    # Standard deviation
    data['BB_Std'] = data['Close'].rolling(window=window).std()
    
    # Upper and lower bands
    data['BB_Upper'] = data['BB_Middle'] + (num_std * data['BB_Std'])
    data['BB_Lower'] = data['BB_Middle'] - (num_std * data['BB_Std'])
    
    # %B indicator (position within bands)
    data['BB_Percent'] = (data['Close'] - data['BB_Lower']) / (data['BB_Upper'] - data['BB_Lower'])
    
    # Bandwidth (volatility measure)
    data['BB_Bandwidth'] = (data['BB_Upper'] - data['BB_Lower']) / data['BB_Middle']
    
    return data

data = calculate_bollinger_bands(data)
print("Bollinger Bands calculated successfully!")

Bollinger Bands calculated successfully!


## 5. Average True Range (ATR)

The Average True Range (ATR) is a volatility indicator that measures the average range of price movements over a specified period.

### Formula
ATR is calculated as follows:
1. Calculate True Range (TR) for each period:
   $$
   TR = \max(\text{High} - \text{Low}, |\text{High} - \text{Close}_{\text{prev}}|, |\text{Low} - \text{Close}_{\text{prev}}|)
   $$
2. Calculate the moving average of TR:
   $$
   ATR = \text{SMA}(TR, n)
   $$
   Where $n$ is typically 14 periods.

In [6]:
def calculate_atr(data, window=14):
    """
    Calculate Average True Range (ATR)
    """
    # Calculate True Range components
    data['High_Low'] = data['High'] - data['Low']
    data['High_Close_Prev'] = abs(data['High'] - data['Close'].shift(1))
    data['Low_Close_Prev'] = abs(data['Low'] - data['Close'].shift(1))
    
    # True Range is the maximum of the three
    data['True_Range'] = data[['High_Low', 'High_Close_Prev', 'Low_Close_Prev']].max(axis=1)
    
    # ATR is the moving average of True Range
    data['ATR'] = data['True_Range'].rolling(window=window).mean()
    
    # ATR as percentage of price (normalized ATR)
    data['ATR_Percent'] = (data['ATR'] / data['Close']) * 100
    
    # Clean up temporary columns
    data.drop(['High_Low', 'High_Close_Prev', 'Low_Close_Prev'], axis=1, inplace=True)
    
    return data

data = calculate_atr(data)
print("ATR calculated successfully!")

ATR calculated successfully!


## 6. Volatility Analysis

Volatility is a statistical measure of the dispersion of returns for a given security or market index. It represents the degree of variation of trading prices over time and is a key component in risk assessment and option pricing.

### Key Volatility Measures

#### 6.1 Historical Volatility (Close-to-Close)
The most common volatility measure using closing prices:

$$
\sigma_{\text{historical}} = \sqrt{\frac{252}{n-1} \sum_{i=1}^{n} (r_i - \bar{r})^2}
$$

Where:
- $r_i = \ln(P_i / P_{i-1})$ are the log returns
- $\bar{r}$ is the mean return over the period
- $n$ is the number of observations
- $252$ is the annualization factor (trading days per year)

#### 6.2 Parkinson Volatility Estimator
A more efficient estimator that uses high and low prices, providing better estimates with the same data:

$$
\sigma^2_{\text{Parkinson}} = \frac{1}{4\ln(2)} \left[\ln\left(\frac{H_t}{L_t}\right)\right]^2
$$

Where $H_t$ and $L_t$ are the High and Low prices respectively.

### Volatility Properties

1. **Volatility Clustering**: High volatility periods tend to be followed by high volatility periods
2. **Mean Reversion**: Volatility tends to revert to its long-term average
3. **Asymmetry**: Negative returns often lead to higher volatility than positive returns (leverage effect)

Let's calculate the most commonly used volatility measures and their practical applications.

In [ ]:
def calculate_volatility_measures(data, window=20):
    """
    Calculate the most common volatility measures
    """
    # Calculate returns
    data['Returns'] = data['Close'].pct_change()
    
    # 1. Historical Volatility (Close-to-Close) - Most common approach
    data['Historical_Vol'] = data['Returns'].rolling(window=window).std() * np.sqrt(252) * 100
    
    # 2. Parkinson Volatility Estimator (High-Low) - More efficient alternative
    # Uses intraday range information for better volatility estimation
    parkinson_vol = (1 / (4 * np.log(2))) * (np.log(data['High'] / data['Low'])) ** 2
    data['Parkinson_Vol'] = np.sqrt(parkinson_vol.rolling(window=window).mean() * 252) * 100
    
    # Volatility percentile (where current volatility ranks historically)
    data['Vol_Percentile'] = data['Historical_Vol'].rolling(window=252).rank(pct=True) * 100
    
    # Volatility Z-Score (standardized volatility)
    vol_mean = data['Historical_Vol'].rolling(window=252).mean()
    vol_std = data['Historical_Vol'].rolling(window=252).std()
    data['Vol_ZScore'] = (data['Historical_Vol'] - vol_mean) / vol_std
    
    # Volatility trend (is volatility increasing or decreasing?)
    data['Vol_Trend'] = data['Historical_Vol'].rolling(window=5).mean() / data['Historical_Vol'].rolling(window=20).mean()
    
    # Volatility regime classification
    def classify_vol_regime(vol_percentile):
        if pd.isna(vol_percentile):
            return 'Unknown'
        elif vol_percentile < 20:
            return 'Low Vol'
        elif vol_percentile < 40:
            return 'Below Average'
        elif vol_percentile < 60:
            return 'Average'
        elif vol_percentile < 80:
            return 'Above Average'
        else:
            return 'High Vol'
    
    data['Vol_Regime'] = data['Vol_Percentile'].apply(classify_vol_regime)
    
    return data

data = calculate_volatility_measures(data)
print("Volatility measures calculated successfully!")

# Compare the two main volatility estimators
latest = data.iloc[-1]
print(f"\nVolatility Estimator Comparison (Latest Values):")
print(f"Historical Volatility (Close-to-Close): {latest['Historical_Vol']:.2f}%")
print(f"Parkinson Volatility (High-Low):        {latest['Parkinson_Vol']:.2f}%")

print(f"\nVolatility Analysis:")
print(f"Volatility Percentile: {latest['Vol_Percentile']:.1f}%")
print(f"Volatility Z-Score: {latest['Vol_ZScore']:.2f}")
print(f"Volatility Trend: {latest['Vol_Trend']:.2f} ({'Increasing' if latest['Vol_Trend'] > 1 else 'Decreasing'})")
print(f"Volatility Regime: {latest['Vol_Regime']}")

# Display recent volatility statistics
recent_vol = data['Historical_Vol'].dropna().tail(1).iloc[0]
vol_percentile = data['Vol_Percentile'].dropna().tail(1).iloc[0]
print(f"\nSummary:")
print(f"Current Historical Volatility: {recent_vol:.2f}%")
print(f"Volatility Percentile: {vol_percentile:.1f}%")

Enhanced volatility measures calculated successfully!

Volatility Estimator Comparison (Latest Values):
Historical (Close-Close): 17.48%
Parkinson (High-Low):    19.08%
Garman-Klass (OHLC):     21.45%
Rogers-Satchell:         21.45%
Yang-Zhang (with gaps):  21.94%

Volatility Analysis:
Volatility Percentile: 18.3%
Volatility Z-Score: -0.66
Volatility Trend: 0.95 (Decreasing)
Volatility Regime: Low Vol

Summary:
Current Historical Volatility: 17.48%
Volatility Percentile: 18.3%


## 7. VIX Analysis (Fear & Greed Index)

The VIX (Volatility Index), often called the "fear gauge," measures the market's expectation of volatility over the next 30 days. It's derived from S&P 500 index options and represents the market's consensus view of future volatility.

### VIX Calculation Methodology

The VIX is calculated using a complex formula that incorporates the prices of multiple SPX options:

$$
VIX = 100 \times \sqrt{\frac{2}{T} \sum_i \frac{\Delta K_i}{K_i^2} e^{RT} Q(K_i) - \frac{1}{T}\left[\frac{F}{K_0} - 1\right]^2}
$$

Where:
- $T$ = Time to expiration (30 days for VIX)
- $F$ = Forward index level derived from index option prices
- $K_0$ = Strike price at which $F \geq K_0$
- $K_i$ = Strike price of the $i^{th}$ out-of-the-money option
- $\Delta K_i$ = Interval between strike prices
- $R$ = Risk-free interest rate
- $Q(K_i)$ = Mid-quote of the option with strike $K_i$

### VIX Interpretation Levels

**Traditional VIX Levels:**
- **Below 12**: Extreme complacency, potential for volatility expansion
- **12-20**: Normal market conditions, moderate fear/greed
- **20-30**: Elevated fear, increased market uncertainty
- **Above 30**: Extreme fear, potential market stress or crisis
- **Above 40**: Panic conditions, major market dislocations

### VIX Term Structure

The VIX term structure shows implied volatility across different time horizons:

$$
\text{Term Structure Slope} = \frac{VIX9D - VIX}{VIX30D - VIX9D}
$$

- **Contango**: VIX < VIX3M (normal condition, volatility expected to increase)
- **Backwardation**: VIX > VIX3M (stressed condition, volatility expected to decrease)

### VIX Trading Strategies

1. **Mean Reversion**: VIX tends to revert to its long-term average (~19-20)
2. **Volatility Risk Premium**: VIX typically trades above realized volatility
3. **Crisis Alpha**: VIX spikes during market stress, providing portfolio hedging

### Mathematical Relationships

**VIX vs. S&P 500 Correlation:**
$$
\rho_{VIX, SPX} = \text{Corr}(\Delta VIX, \Delta SPX) \approx -0.7
$$

**Volatility Risk Premium:**
$$
VRP = IV_{30d} - RV_{30d}
$$
Where $IV$ is implied volatility (VIX) and $RV$ is realized volatility.

The VIX (Volatility Index) measures market's expectation of volatility. Let's fetch VIX data and analyze its relationship with our stock.

In [13]:
# Fetch VIX data and related volatility indices
try:
    # Fetch VIX (30-day implied volatility)
    vix_ticker = yf.Ticker('^VIX')
    vix_ticker.session = session
    vix_data = vix_ticker.history(start=start_date, end=end_date, interval='1d')
    
    # Fetch VIX9D (9-day implied volatility) if available
    try:
        vix9d_ticker = yf.Ticker('^VIX9D')
        vix9d_ticker.session = session
        vix9d_data = vix9d_ticker.history(start=start_date, end=end_date, interval='1d')
        data['VIX9D'] = vix9d_data['Close'].reindex(data.index)
    except:
        print("VIX9D data not available")
        data['VIX9D'] = np.nan
    
    # Fetch VIX3M (3-month implied volatility) if available
    try:
        vix3m_ticker = yf.Ticker('^VIX3M')
        vix3m_ticker.session = session
        vix3m_data = vix3m_ticker.history(start=start_date, end=end_date, interval='1d')
        data['VIX3M'] = vix3m_data['Close'].reindex(data.index)
    except:
        print("VIX3M data not available")
        data['VIX3M'] = np.nan
    
    # Main VIX data
    data['VIX'] = vix_data['Close'].reindex(data.index)
    
    # VIX interpretation levels and regimes
    data['VIX_Regime'] = pd.cut(data['VIX'], 
                               bins=[0, 12, 20, 30, 40, 100], 
                               labels=['Complacency', 'Normal', 'Elevated Fear', 'High Fear', 'Panic'])
    
    # Calculate VIX-based indicators
    
    # 1. VIX Term Structure (if VIX3M available)
    if not data['VIX3M'].isna().all():
        data['VIX_Term_Structure'] = data['VIX'] - data['VIX3M']
        data['VIX_Contango'] = data['VIX_Term_Structure'] < 0  # VIX below VIX3M (normal)
        data['VIX_Backwardation'] = data['VIX_Term_Structure'] > 5  # VIX significantly above VIX3M (stress)
    
    # 2. Volatility Risk Premium (VIX vs Realized Volatility)
    # Calculate 30-day realized volatility
    data['Realized_Vol_30D'] = data['Returns'].rolling(window=30).std() * np.sqrt(252) * 100
    data['Vol_Risk_Premium'] = data['VIX'] - data['Realized_Vol_30D']
    
    # 3. VIX momentum and mean reversion indicators
    vix_sma_20 = data['VIX'].rolling(window=20).mean()
    data['VIX_Mean_Reversion'] = (data['VIX'] - vix_sma_20) / vix_sma_20 * 100
    data['VIX_Momentum'] = data['VIX'].pct_change(periods=5) * 100  # 5-day VIX change
    
    # 4. VIX percentile ranking
    data['VIX_Percentile'] = data['VIX'].rolling(window=252).rank(pct=True) * 100
    
    # 5. VIX spike detection
    vix_rolling_mean = data['VIX'].rolling(window=20).mean()
    vix_rolling_std = data['VIX'].rolling(window=20).std()
    data['VIX_Spike'] = (data['VIX'] - vix_rolling_mean) > (2 * vix_rolling_std)
    
    # 6. VIX-SPY correlation (inverse relationship)
    spy_returns = data['Close'].pct_change()
    vix_changes = data['VIX'].pct_change()
    data['VIX_SPY_Correlation'] = spy_returns.rolling(window=20).corr(vix_changes)
    
    print("VIX data and analysis completed successfully!")
    print(f"Current VIX level: {data['VIX'].dropna().iloc[-1]:.2f}")
    print(f"Current VIX regime: {data['VIX_Regime'].dropna().iloc[-1]}")
    print(f"VIX Percentile: {data['VIX_Percentile'].dropna().iloc[-1]:.1f}%")
    
    if not data['Vol_Risk_Premium'].isna().all():
        current_vrp = data['Vol_Risk_Premium'].dropna().iloc[-1]
        print(f"Volatility Risk Premium: {current_vrp:.2f}% ({'Positive' if current_vrp > 0 else 'Negative'})")
    
    if not data['VIX_Term_Structure'].isna().all():
        current_ts = data['VIX_Term_Structure'].dropna().iloc[-1]
        ts_state = "Contango" if current_ts < 0 else "Backwardation" if current_ts > 5 else "Normal"
        print(f"VIX Term Structure: {current_ts:.2f} ({ts_state})")
    
    # Recent VIX statistics
    recent_vix_stats = {
        'Current': data['VIX'].dropna().iloc[-1],
        '20-day Average': data['VIX'].dropna().tail(20).mean(),
        '1-year Average': data['VIX'].dropna().tail(252).mean(),
        'Recent High (20d)': data['VIX'].dropna().tail(20).max(),
        'Recent Low (20d)': data['VIX'].dropna().tail(20).min()
    }
    
    print(f"\nVIX Statistics Summary:")
    for key, value in recent_vix_stats.items():
        print(f"  {key}: {value:.2f}")
        
except Exception as e:
    print(f"Could not fetch VIX data: {e}")
    data['VIX'] = np.nan
    data['VIX_Regime'] = np.nan

VIX data and analysis completed successfully!
Could not fetch VIX data: single positional indexer is out-of-bounds


## 8. Comprehensive Visualization

Let's create a comprehensive dashboard showing all our volume and volatility indicators.

In [14]:
# Create comprehensive visualization
fig = make_subplots(
    rows=6, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.02,
    horizontal_spacing=0.05,
    subplot_titles=(
        f'{symbol} Price with Bollinger Bands', 'Volume Profile',
        'Volume Analysis', 'On-Balance Volume (OBV)',
        'Bollinger Band %B', 'Average True Range (ATR)',
        'Historical Volatility', 'VIX (Market Fear)',
        'Volume Ratio', 'Volatility Percentile',
        'VWAP Analysis', 'ATR Percentage'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
    row_heights=[0.25, 0.15, 0.15, 0.15, 0.15, 0.15]
)

# 1. Price with Bollinger Bands
fig.add_trace(go.Candlestick(
    x=data.index, open=data['Open'], high=data['High'],
    low=data['Low'], close=data['Close'], name='Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data.index, y=data['BB_Upper'], name='BB Upper',
    line=dict(color='red', dash='dash')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data.index, y=data['BB_Middle'], name='BB Middle',
    line=dict(color='blue')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=data.index, y=data['BB_Lower'], name='BB Lower',
    line=dict(color='green', dash='dash')
), row=1, col=1)

# 2. Volume Profile (horizontal bar chart)
fig.add_trace(go.Bar(
    y=volume_profile['price_level'], 
    x=volume_profile['volume'],
    orientation='h',
    name='Volume Profile',
    marker_color='lightblue'
), row=1, col=2)

# Add POC and Value Area lines
fig.add_hline(y=poc_price, row=1, col=2, line_dash="solid", line_color="red", annotation_text="POC")
fig.add_hline(y=va_high, row=1, col=2, line_dash="dash", line_color="orange", annotation_text="VA High")
fig.add_hline(y=va_low, row=1, col=2, line_dash="dash", line_color="orange", annotation_text="VA Low")

# 3. Volume Analysis
fig.add_trace(go.Bar(
    x=data.index, y=data['Volume'], name='Volume',
    marker_color='lightgray'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=data.index, y=data['Volume_SMA_10'], name='Vol SMA 10',
    line=dict(color='orange')
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=data.index, y=data['Volume_SMA_50'], name='Vol SMA 50',
    line=dict(color='red')
), row=2, col=1)

# 4. On-Balance Volume
fig.add_trace(go.Scatter(
    x=data.index, y=data['OBV'], name='OBV',
    line=dict(color='purple')
), row=2, col=2)

# 5. Bollinger Band %B
fig.add_trace(go.Scatter(
    x=data.index, y=data['BB_Percent'], name='%B',
    line=dict(color='blue')
), row=3, col=1)
fig.add_hline(y=1, row=3, col=1, line_dash="dash", line_color="red")
fig.add_hline(y=0, row=3, col=1, line_dash="dash", line_color="green")
fig.add_hline(y=0.5, row=3, col=1, line_dash="dot", line_color="gray")

# 6. Average True Range
fig.add_trace(go.Scatter(
    x=data.index, y=data['ATR'], name='ATR',
    line=dict(color='red')
), row=3, col=2)

# 7. Historical Volatility
fig.add_trace(go.Scatter(
    x=data.index, y=data['Historical_Vol'], name='Hist Vol',
    line=dict(color='orange')
), row=4, col=1)

# 8. VIX (if available)
if not data['VIX'].isna().all():
    fig.add_trace(go.Scatter(
        x=data.index, y=data['VIX'], name='VIX',
        line=dict(color='black')
    ), row=4, col=2)
    fig.add_hline(y=20, row=4, col=2, line_dash="dash", line_color="orange")
    fig.add_hline(y=30, row=4, col=2, line_dash="dash", line_color="red")

# 9. Volume Ratio
fig.add_trace(go.Scatter(
    x=data.index, y=data['Volume_Ratio'], name='Vol Ratio',
    line=dict(color='green')
), row=5, col=1)
fig.add_hline(y=1, row=5, col=1, line_dash="dash", line_color="gray")
fig.add_hline(y=1.5, row=5, col=1, line_dash="dash", line_color="orange")

# 10. Volatility Percentile
fig.add_trace(go.Scatter(
    x=data.index, y=data['Vol_Percentile'], name='Vol %ile',
    line=dict(color='purple')
), row=5, col=2)
fig.add_hline(y=80, row=5, col=2, line_dash="dash", line_color="red")
fig.add_hline(y=20, row=5, col=2, line_dash="dash", line_color="green")

# 11. VWAP Analysis
fig.add_trace(go.Scatter(
    x=data.index, y=data['Close'], name='Close',
    line=dict(color='blue')
), row=6, col=1)
fig.add_trace(go.Scatter(
    x=data.index, y=data['VWAP'], name='VWAP',
    line=dict(color='red', dash='dash')
), row=6, col=1)

# 12. ATR Percentage
fig.add_trace(go.Scatter(
    x=data.index, y=data['ATR_Percent'], name='ATR %',
    line=dict(color='brown')
), row=6, col=2)

# Update layout
fig.update_layout(
    height=1400,
    title_text=f"Volume and Volatility Analysis Dashboard - {symbol}",
    showlegend=False
)

# Remove x-axis range slider for cleaner look
fig.update_xaxes(rangeslider_visible=False)

fig.show()

## 9. Multi-Timeframe Analysis

Let's analyze volatility patterns across different timeframes to get a more comprehensive view.

In [10]:
def multi_timeframe_analysis(symbol, timeframes=['1d', '1wk', '1mo']):
    """
    Analyze volatility across multiple timeframes
    """
    results = {}
    
    for tf in timeframes:
        try:
            # Fetch data for this timeframe
            ticker = yf.Ticker(symbol)
            ticker.session = session
            tf_data = ticker.history(start='2023-01-01', interval=tf)
            
            # Calculate basic volatility metrics
            returns = tf_data['Close'].pct_change().dropna()
            volatility = returns.std() * np.sqrt(252 if tf == '1d' else 52 if tf == '1wk' else 12) * 100
            
            # Calculate ATR for this timeframe
            tf_data = calculate_atr(tf_data)
            current_atr = tf_data['ATR'].iloc[-1]
            
            results[tf] = {
                'volatility': volatility,
                'current_atr': current_atr,
                'atr_percent': (current_atr / tf_data['Close'].iloc[-1]) * 100,
                'data_points': len(tf_data)
            }
            
        except Exception as e:
            print(f"Error fetching {tf} data: {e}")
            continue
    
    return results

# Perform multi-timeframe analysis
mtf_results = multi_timeframe_analysis(symbol)

# Display results
print("Multi-Timeframe Volatility Analysis")
print("=" * 40)
for tf, metrics in mtf_results.items():
    print(f"\n{tf.upper()} Timeframe:")
    print(f"  Annualized Volatility: {metrics['volatility']:.2f}%")
    print(f"  Current ATR: ${metrics['current_atr']:.2f}")
    print(f"  ATR as % of Price: {metrics['atr_percent']:.2f}%")
    print(f"  Data Points: {metrics['data_points']}")

Multi-Timeframe Volatility Analysis

1D Timeframe:
  Annualized Volatility: 26.25%
  Current ATR: $4.22
  ATR as % of Price: 2.01%
  Data Points: 635

1WK Timeframe:
  Annualized Volatility: 25.66%
  Current ATR: $11.67
  ATR as % of Price: 5.55%
  Data Points: 133

1MO Timeframe:
  Annualized Volatility: 20.45%
  Current ATR: $25.63
  ATR as % of Price: 12.20%
  Data Points: 31


## 10. Trading Signals and Interpretation

Let's create some practical trading signals based on our volume and volatility indicators.

In [11]:
def generate_volume_volatility_signals(data):
    """
    Generate trading signals based on volume and volatility indicators
    """
    signals = pd.DataFrame(index=data.index)
    
    # Bollinger Band signals
    signals['BB_Oversold'] = data['BB_Percent'] < 0.1  # Below lower band
    signals['BB_Overbought'] = data['BB_Percent'] > 0.9  # Above upper band
    signals['BB_Squeeze'] = data['BB_Bandwidth'] < data['BB_Bandwidth'].rolling(20).quantile(0.2)
    
    # Volume signals
    signals['High_Volume'] = data['Volume_Ratio'] > 1.5  # 50% above average
    signals['Volume_Breakout'] = (data['Volume_Ratio'] > 2) & (data['Close'] > data['Close'].shift(1))
    
    # Volatility signals
    signals['Low_Vol_Environment'] = data['Vol_Percentile'] < 20
    signals['High_Vol_Environment'] = data['Vol_Percentile'] > 80
    signals['Vol_Expansion'] = data['ATR'] > data['ATR'].rolling(20).mean() * 1.2
    
    # VIX signals (if available)
    if not data['VIX'].isna().all():
        signals['VIX_Fear'] = data['VIX'] > 30  # Extreme fear
        signals['VIX_Complacency'] = data['VIX'] < 12  # Low fear
    
    # VWAP signals
    signals['Above_VWAP'] = data['Close'] > data['VWAP']
    signals['VWAP_Support'] = (data['Low'] <= data['VWAP']) & (data['Close'] > data['VWAP'])
    
    # Composite signals
    signals['Bullish_Setup'] = (
        signals['BB_Oversold'] & 
        signals['High_Volume'] & 
        ~signals['High_Vol_Environment']
    )
    
    signals['Bearish_Setup'] = (
        signals['BB_Overbought'] & 
        signals['High_Volume'] & 
        signals['High_Vol_Environment']
    )
    
    signals['Breakout_Setup'] = (
        signals['BB_Squeeze'] & 
        signals['Low_Vol_Environment'] & 
        signals['Volume_Breakout']
    )
    
    return signals

# Generate signals
signals = generate_volume_volatility_signals(data)

# Count recent signals
recent_signals = signals.tail(20)
print("Recent Signal Summary (Last 20 days):")
print("=" * 40)
for col in ['Bullish_Setup', 'Bearish_Setup', 'Breakout_Setup', 'BB_Squeeze', 'High_Volume']:
    count = recent_signals[col].sum()
    print(f"{col}: {count} occurrences")

# Show latest signal status
latest = signals.iloc[-1]
print("\nCurrent Signal Status:")
print("=" * 25)
active_signals = [col for col in signals.columns if latest[col]]
if active_signals:
    for signal in active_signals:
        print(f"✓ {signal}")
else:
    print("No active signals")

Recent Signal Summary (Last 20 days):
Bullish_Setup: 0 occurrences
Bearish_Setup: 0 occurrences
Breakout_Setup: 0 occurrences
BB_Squeeze: 8 occurrences
High_Volume: 2 occurrences

Current Signal Status:
✓ Low_Vol_Environment
✓ Above_VWAP


## 11. Risk Management with Volatility

Let's demonstrate how to use volatility indicators for risk management and position sizing.

In [ ]:
def calculate_position_sizing(data, account_balance=100000, risk_per_trade=0.02, confidence_level=0.95):
    """
    Calculate position sizes based on volatility and risk management principles
    """
    results = pd.DataFrame(index=data.index)
    
    # Risk amount per trade
    risk_amount = account_balance * risk_per_trade
    
    # ATR-based stop loss (2x ATR)
    results['Stop_Loss_Distance'] = 2 * data['ATR']
    
    # Position size based on ATR stop
    results['Position_Size_ATR'] = risk_amount / results['Stop_Loss_Distance']
    
    # Volatility-adjusted position size
    avg_vol = data['Historical_Vol'].rolling(252).mean()
    vol_adjustment = avg_vol / data['Historical_Vol']
    results['Position_Size_Vol_Adj'] = results['Position_Size_ATR'] * vol_adjustment
    
    # Value at Risk (VaR) based position sizing
    returns = data['Returns'].dropna()
    var_1day = returns.rolling(252).quantile(1 - confidence_level)
    results['Max_Position_VaR'] = (risk_amount / abs(var_1day)) / data['Close']
    
    # Kelly Criterion (simplified)
    win_rate = 0.55  # Assumed win rate
    avg_win = data['Returns'].where(data['Returns'] > 0).rolling(252).mean()
    avg_loss = abs(data['Returns'].where(data['Returns'] < 0).rolling(252).mean())
    kelly_f = (win_rate * avg_win - (1 - win_rate) * avg_loss) / avg_win
    kelly_f = kelly_f.clip(0, 0.25)  # Cap at 25% of account
    results['Kelly_Position'] = (account_balance * kelly_f) / data['Close']
    
    # Final recommended position (conservative approach)
    results['Recommended_Shares'] = results[[
        'Position_Size_ATR', 'Position_Size_Vol_Adj', 'Max_Position_VaR'
    ]].min(axis=1)
    
    results['Position_Value'] = results['Recommended_Shares'] * data['Close']
    results['Portfolio_Allocation'] = results['Position_Value'] / account_balance * 100
    
    return results

# Calculate position sizing
position_data = calculate_position_sizing(data)

# Display recent position sizing recommendations
recent_positions = position_data.tail(5)
print("Position Sizing Recommendations (Last 5 days):")
print("=" * 50)
print(f"{'Date':<12} {'Shares':<8} {'Value':<10} {'Stop Loss':<10} {'% Portfolio':<12}")
print("-" * 50)

for idx, row in recent_positions.iterrows():
    if pd.notna(row['Recommended_Shares']):
        print(f"{idx.strftime('%Y-%m-%d'):<12} "
              f"{row['Recommended_Shares']:<8.0f} "
              f"${row['Position_Value']:<9.0f} "
              f"${row['Stop_Loss_Distance']:<9.2f} "
              f"{row['Portfolio_Allocation']:<11.1f}%")

# Current recommendations
current = position_data.iloc[-1]
if pd.notna(current['Recommended_Shares']):
    print(f"\nCurrent Recommendation:")
    print(f"  Recommended Shares: {current['Recommended_Shares']:.0f}")
    print(f"  Position Value: ${current['Position_Value']:.0f}")
    print(f"  Stop Loss Distance: ${current['Stop_Loss_Distance']:.2f}")
    print(f"  Portfolio Allocation: {current['Portfolio_Allocation']:.1f}%")

## 12. Key Insights and Summary

Let's summarize the key insights from our volume and volatility analysis.

In [ ]:
def generate_analysis_summary(data, signals, position_data, symbol):
    """
    Generate a comprehensive analysis summary
    """
    # Get latest values
    latest = data.iloc[-1]
    latest_signals = signals.iloc[-1]
    
    print(f"Volume and Volatility Analysis Summary for {symbol}")
    print("=" * 60)
    
    # Current market metrics
    print("\n📊 Current Market Metrics:")
    print(f"  Current Price: ${latest['Close']:.2f}")
    print(f"  Historical Volatility: {latest['Historical_Vol']:.1f}%")
    print(f"  Volatility Percentile: {latest['Vol_Percentile']:.0f}%")
    print(f"  ATR: ${latest['ATR']:.2f} ({latest['ATR_Percent']:.1f}% of price)")
    
    # Bollinger Bands status
    print("\n📈 Bollinger Bands Analysis:")
    bb_position = latest['BB_Percent']
    if bb_position > 0.8:
        bb_status = "Near upper band (overbought)"
    elif bb_position < 0.2:
        bb_status = "Near lower band (oversold)"
    else:
        bb_status = "Within normal range"
    print(f"  Position: {bb_position:.2f} - {bb_status}")
    print(f"  Bandwidth: {latest['BB_Bandwidth']:.4f}")
    
    # Volume analysis
    print("\n📦 Volume Analysis:")
    vol_status = "High" if latest['Volume_Ratio'] > 1.5 else "Normal" if latest['Volume_Ratio'] > 0.5 else "Low"
    print(f"  Current Volume vs Average: {latest['Volume_Ratio']:.1f}x ({vol_status})")
    print(f"  Price vs VWAP: {'Above' if latest['Close'] > latest['VWAP'] else 'Below'} (${latest['VWAP']:.2f})")
    
    # VIX analysis (if available)
    if not pd.isna(latest['VIX']):
        print("\n😰 Market Fear Analysis (VIX):")
        vix_level = latest['VIX']
        if vix_level > 30:
            vix_status = "Extreme fear"
        elif vix_level > 20:
            vix_status = "Elevated fear"
        elif vix_level < 12:
            vix_status = "Complacency"
        else:
            vix_status = "Normal"
        print(f"  VIX Level: {vix_level:.1f} ({vix_status})")
    
    # Active signals
    print("\n🚦 Active Signals:")
    active_signals = [col for col in signals.columns if latest_signals[col] and col.endswith('_Setup')]
    if active_signals:
        for signal in active_signals:
            print(f"  ✓ {signal.replace('_', ' ')}")
    else:
        print("  No major setup signals active")
    
    # Risk assessment
    print("\n⚠️ Risk Assessment:")
    risk_level = "High" if latest['Vol_Percentile'] > 80 else "Low" if latest['Vol_Percentile'] < 20 else "Medium"
    print(f"  Volatility Risk: {risk_level}")
    
    if not pd.isna(position_data.iloc[-1]['Recommended_Shares']):
        recommended_allocation = position_data.iloc[-1]['Portfolio_Allocation']
        print(f"  Recommended Allocation: {recommended_allocation:.1f}% of portfolio")
    
    # Trading recommendations
    print("\n💡 Trading Recommendations:")
    if latest_signals['BB_Squeeze'] and latest['Vol_Percentile'] < 30:
        print("  📈 Watch for breakout - low volatility compression detected")
    
    if latest_signals['High_Volume'] and latest['Close'] > latest['VWAP']:
        print("  📈 Bullish volume confirmation above VWAP")
    
    if latest['BB_Percent'] < 0.1 and not latest_signals['High_Vol_Environment']:
        print("  📈 Potential oversold bounce opportunity")
    
    if latest['BB_Percent'] > 0.9 and latest_signals['High_Vol_Environment']:
        print("  📉 Potential overbought reversal risk")
    
    print("\n" + "=" * 60)
    print("Note: This analysis is for educational purposes only. Always conduct your own research.")

# Generate summary
generate_analysis_summary(data, signals, position_data, symbol)

## Conclusion

In this notebook, we've explored comprehensive volume and volatility analysis including:

### Key Concepts Learned:

1. **Volume Analysis**:
   - Volume moving averages and ratios
   - On-Balance Volume (OBV) for trend confirmation
   - Volume-Weighted Average Price (VWAP) as dynamic support/resistance
   - Volume Profile for identifying key price levels

2. **Volatility Indicators**:
   - Bollinger Bands for overbought/oversold conditions
   - Average True Range (ATR) for volatility measurement
   - Historical volatility and percentile rankings
   - VIX analysis for market sentiment

3. **Multi-Timeframe Analysis**:
   - Comparing volatility across different timeframes
   - Understanding how volatility scales with time

4. **Risk Management Applications**:
   - ATR-based stop losses
   - Volatility-adjusted position sizing
   - Value at Risk (VaR) calculations

### Practical Applications:

- **Entry Timing**: Use volume confirmation with price breakouts
- **Risk Management**: Adjust position sizes based on current volatility
- **Market Regime Detection**: Identify low/high volatility environments
- **Support/Resistance**: Use VWAP and volume profile for key levels

### Next Steps:

The next module will focus on creating custom indicators and combining multiple signals for robust trading systems. We'll also explore machine learning applications to technical analysis.